In [1]:
import csv
import re

In [2]:
value = "$480,000"

value = value.encode('ascii', errors='ignore').decode() # Filter out non-ascii characters for now

# Remove percent
value = re.sub(r'[%\*]', '', value)

value = re.sub(r'[\$,\.\*]', '', value)
print(value)

480000


In [3]:
pure_f = open('/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/AmtIssued_Cupon_July_22_CLEAN.csv', 'w+')

pure = csv.writer(pure_f)

impure_f = open('/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/AmtIssued_Cupon_July_22_BAD.csv', 'w+')
impure = csv.writer(impure_f)

In [4]:
with open('/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/AmtIssued_Cupon_July_22.otto_ml.muni_tables.csv') as csvfile:
    muni_bonds = csv.reader(csvfile)
    pure.writerow(["file_id","cusip","maturity","amount","interest_rate","price","yield"])
    impure.writerow(["file_id","cusip","maturity","amount","interest_rate","price","yield"])
    for row in muni_bonds:
        for idx, cell in enumerate(row):
            if cell == "NaN" or cell == '-':
                row[idx] = None
        row[1] = row[1].replace(' ', '')
        amt = row[3]
        int_rate = row[4]
        price = row[5]
        y = row[6]
        is_pure = True

        if amt:
            amt = amt.encode('ascii', errors='ignore').decode()
            amt = re.sub(r'[-@_\$\* Cc,ab†]|(\(.\))', '', amt)
            if re.sub(r'[\.\*]', '', amt).isnumeric():
                row[3] = amt
            else:
                is_pure = False
        
        if int_rate:
            int_rate = int_rate.encode('ascii', errors='ignore').decode()
            int_rate = re.sub(r'[-@_%\* Ccab†]|(\(.\))', '', int_rate)
            if re.sub(r'[%\*\.]', '', int_rate).isnumeric():
                row[4] = int_rate
            else:
                is_pure = False
        
        if price:
            price = price.encode('ascii', errors='ignore').decode()
            price = re.sub(r'[-@_%\* Ccab†]|(\(.\))', '', price)
            if re.sub(r'[%\*\.]', '', price).isnumeric():
                row[5] = price
            else:
                is_pure = False
        
        if y:
            y = y.encode('ascii', errors='ignore').decode()
            y = re.sub(r'[-@_%\* Ccab†]|(\(.\))', '', y)
            y = y.replace('(†)', '')
            if re.sub(r'[%\*\.]', '', y).isnumeric():
                row[6] = y
            else:
                is_pure = False
        
        if is_pure:
            pure.writerow(row)
        else:
            impure.writerow(row)
    


In [5]:
pure_f.close()
impure_f.close()